In [1]:
%matplotlib inline

In [2]:
from io import open
import unicodedata
import string
import re
import random

import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
data_dir = '/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/'

In [5]:
data_dir

'/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/'

In [5]:
!wget https://manythings.org/anki/rus-eng.zip
# !unzip rus-eng.zip
!unzip rus-eng.zip -d data_dir

--2021-03-06 19:14:51--  https://manythings.org/anki/rus-eng.zip
Resolving manythings.org (manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3031::6815:37de, ...
Connecting to manythings.org (manythings.org)|104.21.55.222|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: http://www.manythings.org/anki/rus-eng.zip [following]
--2021-03-06 19:14:51--  http://www.manythings.org/anki/rus-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.21.55.222, 172.67.173.198, 2606:4700:3036::ac43:adc6, ...
Connecting to www.manythings.org (www.manythings.org)|104.21.55.222|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14042100 (13M) [application/zip]
Saving to: ‘rus-eng.zip.1’

rus-eng.zip.1       100%[===================>]  13.39M  21.9MB/s    in 0.6s    

2021-03-06 19:14:52 (21.9 MB/s) - ‘rus-eng.zip.1’ saved [14042100/14042100]

Archive:  rus-eng.zip
  inflating: data_dir/rus.txt        
  inflating: data_d

In [6]:
!tail '/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/eng-rus.txt'

Tom said, "You can kiss your girlfriend goodbye if you don't kiss her goodbye," which meant, "If you don't kiss your girlfriend goodbye, then you'll never see her again."	«Можешь попрощаться со своей подружкой, если ты с ней не попрощаешься», — сказал Том, что означало «если ты не попрощаешься со своей подружкой, то больше ты её никогда не увидишь».	CC-BY 2.0 (France) Attribution: tatoeba.org #1065032 (CK) & #4435211 (sharptoothed)
The more countries a language is spoken in, the less important it is to sound like a native speaker, since speakers of that language are accustomed to hearing various dialects.	Чем в большем количестве стран используется тот или иной язык, тем менее важно иметь такое же произношение, как у его носителей, так как носители этого языка привыкли к звучанию различных акцентов.	CC-BY 2.0 (France) Attribution: tatoeba.org #954354 (CK) & #4465953 (Wezel)
A mistake young people often make is to start learning too many languages at the same time, as they underestimate

In [7]:
import pandas as pd

Здесь мешает в конце всякая реклама.

In [8]:
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/eng-rus.txt', sep='\t', header=None).drop(2, axis=1)

In [9]:
df.tail(5)


,0,1
421760,"In today's world, we have to equip all our kid...",В современном мире перед нами стоит задача дат...
421761,Death is something that we're often discourage...,"Смерть - это зачастую то, разговоры или даже м..."
421762,"At a moment when our economy is growing, our b...","В тот момент, когда наша экономика растёт, наш..."
421763,Since there are usually multiple websites on a...,"Поскольку сайтов, посвящённых какой-либо теме,..."
421764,Doubtless there exists in this world precisely...,"Несомненно, для каждого мужчины в этом мире гд..."


In [12]:
#tfile = open('/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/eng_rus.txt', 'a')
#tfile.write(df.to_string())
#tfile.close()

In [10]:
df.to_csv('/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/eng_rus.txt', header=False, index=False, sep='\t', mode='a')


In [11]:
!tail '/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/eng_rus.txt'

"Tom said, ""You can kiss your girlfriend goodbye if you don't kiss her goodbye,"" which meant, ""If you don't kiss your girlfriend goodbye, then you'll never see her again."""	«Можешь попрощаться со своей подружкой, если ты с ней не попрощаешься», — сказал Том, что означало «если ты не попрощаешься со своей подружкой, то больше ты её никогда не увидишь».
The more countries a language is spoken in, the less important it is to sound like a native speaker, since speakers of that language are accustomed to hearing various dialects.	Чем в большем количестве стран используется тот или иной язык, тем менее важно иметь такое же произношение, как у его носителей, так как носители этого языка привыкли к звучанию различных акцентов.
A mistake young people often make is to start learning too many languages at the same time, as they underestimate the difficulties and overestimate their own ability to learn them.	Ошибка, которую часто делают молодые, — начинают учить слишком много языков одновремен

Наконец-то удалили эту вездес.ущую рекламу.

 Теперь точно разделено на английский текст и русский

In [12]:
SOS_token = 0
EOS_token = 1


class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.word2count = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.n_words = 2  # Count SOS and EOS

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)

    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [13]:
# Turn a Unicode string to plain ASCII, thanks to
# http://stackoverflow.com/a/518232/2809427
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters


def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zа-яёA-ZА-ЯЁ.!?]+", r" ", s)
    return s

In [14]:
def readLangs(lang1, lang2, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open('/content/drive/My Drive/Colab Notebooks/eng_rus_seq2seq/%s_%s.txt' % (lang1, lang2), encoding='utf-8').\
        read().strip().split('\n')

    # Split every line into pairs and normalize
    pairs = [[normalizeString(s) for s in l.split('\t')] for l in lines]

    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)

    return input_lang, output_lang, pairs

In [15]:
MAX_LENGTH = 10
eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re "
)


def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1].split(' ')) < MAX_LENGTH and \
        p[1].startswith(eng_prefixes)


def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [16]:
def prepareData(lang1, lang2, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, input_lang.n_words)
    print(output_lang.name, output_lang.n_words)
    return input_lang, output_lang, pairs


input_lang, output_lang, pairs = prepareData('eng', 'rus', True)
print(random.choice(pairs))

Reading lines...
Read 421765 sentence pairs
Trimmed to 24897 sentence pairs
Counting words...
Counted words:
rus 9420
eng 4065
['я лишь говорю тебе правду .', 'i m just telling you the truth .']


The Encoder
-----------





Лекционный EncoderRNN (не запускаю):

In [33]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input, hidden):
        embedded = self.embedding(input).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Добавил параметры 'rnn_type' и 'rnn_n', чтобы было всё в одном месте (по решениям задач).

In [17]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size, rnn_type='gru', rnn_n=1):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_n = rnn_n

        self.embedding = nn.Embedding(input_size, hidden_size)
        if rnn_type == 'gru':
            self.rnn = nn.GRU(hidden_size, hidden_size)  # по умолчанию, как на лекции
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(hidden_size, hidden_size) # меняем gru на lstm
        if self.rnn_n == 2:
            if rnn_type == 'gru':
                self.rnn_2 = nn.GRU(hidden_size, hidden_size) # если два слоя добавляем  gru
            elif rnn_type == 'lstm':
                self.rnn_2 = nn.LSTM(hidden_size, hidden_size) #или добавляем lstm

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output, hidden = self.rnn(output, hidden)
        if self.rnn_n == 2:
            output, hidden = self.rnn_2(output, hidden)
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)

The Decoder
-----------




Лекционный  Decoder (не запускаю):

In [34]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(output_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.gru(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size, device=device)

Добавил параметры 'rnn_type' и 'rnn_n', как и в случае с Encoder.

In [18]:
class DecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, rnn_type='gru', rnn_n=1):
        super(DecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.rnn_n = rnn_n

        self.embedding = nn.Embedding(output_size, hidden_size)
        if rnn_type == 'gru':
            self.rnn = nn.GRU(hidden_size, hidden_size)
        elif rnn_type == 'lstm':
            self.rnn = nn.LSTM(hidden_size, hidden_size)
        if self.rnn_n == 2:
            if rnn_type == 'gru':
                self.rnn_2 = nn.GRU(hidden_size, hidden_size)
            elif rnn_type == 'lstm':
                self.rnn_2 = nn.LSTM(hidden_size, hidden_size)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        output = self.embedding(input).view(1, 1, -1)
        output = F.relu(output)
        output, hidden = self.rnn(output, hidden)
        if self.rnn_n == 2:
            output, hidden = self.rnn_2(output, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden

    def initHidden(self):
        if isinstance(self.rnn, nn.LSTM):
            return (torch.zeros(1, 1, self.hidden_size, device=device),
                    torch.zeros(1, 1, self.hidden_size, device=device))
        return torch.zeros(1, 1, self.hidden_size, device=device)

In [19]:
def indexesFromSentence(lang, sentence):
    return [lang.word2index[word] for word in sentence.split(' ')]


def tensorFromSentence(lang, sentence):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    return torch.tensor(indexes, dtype=torch.long, device=device).view(-1, 1)


def tensorsFromPair(pair):
    input_tensor = tensorFromSentence(input_lang, pair[0])
    target_tensor = tensorFromSentence(output_lang, pair[1])
    return (input_tensor, target_tensor)

In [20]:
teacher_forcing_ratio = 0.5


def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=MAX_LENGTH):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)

    encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[SOS_token]], device=device)

    decoder_hidden = encoder_hidden

    use_teacher_forcing = True if random.random() < teacher_forcing_ratio else False

    if use_teacher_forcing:
        # Teacher forcing: Feed the target as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            loss += criterion(decoder_output, target_tensor[di])
            decoder_input = target_tensor[di]  # Teacher forcing

    else:
        # Without teacher forcing: use its own predictions as the next input
        for di in range(target_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.topk(1)
            decoder_input = topi.squeeze().detach()  # detach from history as input

            loss += criterion(decoder_output, target_tensor[di])
            if decoder_input.item() == EOS_token:
                break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [21]:
import time
import math


def asMinutes(s):
    m = math.floor(s / 60)
    s -= m * 60
    return '%dm %ds' % (m, s)


def timeSince(since, percent):
    now = time.time()
    s = now - since
    es = s / (percent)
    rs = es - s
    return '%s (- %s)' % (asMinutes(s), asMinutes(rs))

In [22]:
def trainIters(encoder, decoder, n_iters, print_every=1000, plot_every=100, learning_rate=0.01):
    start = time.time()
    plot_losses = []
    print_loss_total = 0  # Reset every print_every
    plot_loss_total = 0  # Reset every plot_every

    encoder_optimizer = optim.SGD(encoder.parameters(), lr=learning_rate)
    decoder_optimizer = optim.SGD(decoder.parameters(), lr=learning_rate)
    training_pairs = [tensorsFromPair(random.choice(pairs))
                      for i in range(n_iters)]
    criterion = nn.NLLLoss()

    for iter in range(1, n_iters + 1):
        training_pair = training_pairs[iter - 1]
        input_tensor = training_pair[0]
        target_tensor = training_pair[1]

        loss = train(input_tensor, target_tensor, encoder,
                     decoder, encoder_optimizer, decoder_optimizer, criterion)
        print_loss_total += loss
        plot_loss_total += loss

        if iter % print_every == 0:
            print_loss_avg = print_loss_total / print_every
            print_loss_total = 0
            print('%s (%d %d%%) %.4f' % (timeSince(start, iter / n_iters),
                                         iter, iter / n_iters * 100, print_loss_avg))

        if iter % plot_every == 0:
            plot_loss_avg = plot_loss_total / plot_every
            plot_losses.append(plot_loss_avg)
            plot_loss_total = 0

    showPlot(plot_losses)

In [23]:
import matplotlib.pyplot as plt
plt.switch_backend('agg')
import matplotlib.ticker as ticker
import numpy as np


def showPlot(points):
    plt.figure()
    fig, ax = plt.subplots()
    # this locator puts ticks at regular intervals
    loc = ticker.MultipleLocator(base=0.2)
    ax.yaxis.set_major_locator(loc)
    plt.plot(points)

In [24]:
def evaluate(encoder, decoder, sentence, max_length=MAX_LENGTH):
    with torch.no_grad():
        input_tensor = tensorFromSentence(input_lang, sentence)
        input_length = input_tensor.size()[0]
        encoder_hidden = encoder.initHidden()

        encoder_outputs = torch.zeros(max_length, encoder.hidden_size, device=device)

        for ei in range(input_length):
            encoder_output, encoder_hidden = encoder(input_tensor[ei],
                                                     encoder_hidden)
            encoder_outputs[ei] += encoder_output[0, 0]

        decoder_input = torch.tensor([[SOS_token]], device=device)  # SOS

        decoder_hidden = encoder_hidden

        decoded_words = []

        for di in range(max_length):
            decoder_output, decoder_hidden = decoder(
                decoder_input, decoder_hidden)
            topv, topi = decoder_output.data.topk(1)
            if topi.item() == EOS_token:
                decoded_words.append('<EOS>')
                break
            else:
                decoded_words.append(output_lang.index2word[topi.item()])

            decoder_input = topi.squeeze().detach()

        return decoded_words

In [25]:
def evaluateRandomly(encoder, decoder, n=10):
    for i in range(n):
        pair = random.choice(pairs)
        print('>', pair[0])
        print('=', pair[1])
        output_words = evaluate(encoder, decoder, pair[0])
        output_sentence = ' '.join(output_words)
        print('<', output_sentence)
        print('')

# GRU с одним рекуррентным слоем.

In [26]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 35s (- 22m 19s) (5000 6%) 3.1207
3m 7s (- 20m 15s) (10000 13%) 2.6199
4m 39s (- 18m 38s) (15000 20%) 2.3250
6m 12s (- 17m 3s) (20000 26%) 2.1384
7m 44s (- 15m 29s) (25000 33%) 1.9586
9m 18s (- 13m 57s) (30000 40%) 1.8342
10m 51s (- 12m 24s) (35000 46%) 1.7194
12m 24s (- 10m 51s) (40000 53%) 1.6026
13m 58s (- 9m 18s) (45000 60%) 1.4970
15m 31s (- 7m 45s) (50000 66%) 1.4376
17m 4s (- 6m 12s) (55000 73%) 1.3587
18m 37s (- 4m 39s) (60000 80%) 1.2865
20m 11s (- 3m 6s) (65000 86%) 1.2370
21m 44s (- 1m 33s) (70000 93%) 1.1739
23m 18s (- 0m 0s) (75000 100%) 1.1199


In [27]:
evaluateRandomly(encoder1, decoder1)

> она зависит от мужа .
= she s dependent on her husband .
< she is about her husband . <EOS>

> жаль что вы нас покидаете .
= i m sorry that you re leaving us .
< i m sorry that you re leaving us . <EOS>

> я немного ленив .
= i m a bit lazy .
< i m a little little . <EOS>

> он умнее меня .
= he is more clever than i am .
< he s smarter than me . <EOS>

> она неплохая девушка .
= she s not a bad girl .
< she s not a girl girl . <EOS>

> по понедельникам я всегда дома .
= i m always at home on mondays .
< i m always at home on mondays . <EOS>

> мы рады снова вас видеть .
= we are happy to see you again .
< we re glad to see you again . <EOS>

> я старшии брат тома .
= i m tom s older brother .
< i m tom s older . <EOS>

> я рад что это была ты .
= i m glad it was you .
< i m glad that that it . <EOS>

> ты гораздо быстрее меня .
= you re way faster than me .
< you re much faster than i am . <EOS>



Не плохо, есть совпадения. Не идеально.

# GRU с двумя рекуррентными слоями.

In [28]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, rnn_n=2).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, rnn_n=2).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

2m 8s (- 30m 5s) (5000 6%) 3.2927
4m 12s (- 27m 23s) (10000 13%) 2.8870
6m 18s (- 25m 14s) (15000 20%) 2.7364
8m 24s (- 23m 7s) (20000 26%) 2.5930
10m 30s (- 21m 1s) (25000 33%) 2.4745
12m 37s (- 18m 56s) (30000 40%) 2.3371
14m 43s (- 16m 49s) (35000 46%) 2.2230
16m 50s (- 14m 44s) (40000 53%) 2.1099
18m 57s (- 12m 38s) (45000 60%) 1.9787
21m 4s (- 10m 32s) (50000 66%) 1.8866
23m 11s (- 8m 25s) (55000 73%) 1.8278
25m 18s (- 6m 19s) (60000 80%) 1.7260
27m 25s (- 4m 13s) (65000 86%) 1.6681
29m 32s (- 2m 6s) (70000 93%) 1.5956
31m 39s (- 0m 0s) (75000 100%) 1.5600


In [29]:
evaluateRandomly(encoder1, decoder1)

> я отдаю тебе все что могу .
= i m giving you all i can .
< i m looking for you i can help . <EOS>

> они запачканы .
= they are muddy .
< they are afraid . <EOS>

> он приятныи человек .
= he is an agreeable person .
< he is a kind man . <EOS>

> они не заключенные .
= they re not prisoners .
< they aren t alone . <EOS>

> вы очень хорошии художник .
= you re a very good artist .
< you re a very good liar . <EOS>

> мы не продаем наш дом .
= we re not selling our house .
< we aren t blaming of anything . <EOS>

> удивительно что эта машина еще работает .
= i m surprised this machine still works .
< i m surprised that happened still alive . <EOS>

> он меня на два дюима выше .
= he is two inches taller than i am .
< he is the years older than i am <EOS>

> он упорно готовится к сдаче экзамена .
= he is working hard to pass the examination .
< he is the to for the . . <EOS>

> я прямо позади тебя .
= i m right behind you .
< i m right than you are . <EOS>



Стало хуже. Теряется смысл. То есть увеличение слоев за счёт GRU не желательно.

# LSTM с одним рекуррентным слоем.

In [30]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, rnn_type='lstm').to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, rnn_type='lstm').to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

1m 37s (- 22m 46s) (5000 6%) 3.2181
3m 11s (- 20m 42s) (10000 13%) 2.7291
4m 46s (- 19m 5s) (15000 20%) 2.5222
6m 21s (- 17m 29s) (20000 26%) 2.3335
7m 56s (- 15m 53s) (25000 33%) 2.1593
9m 32s (- 14m 19s) (30000 40%) 2.0555
11m 8s (- 12m 44s) (35000 46%) 1.9385
12m 44s (- 11m 8s) (40000 53%) 1.8397
14m 20s (- 9m 33s) (45000 60%) 1.7686
15m 56s (- 7m 58s) (50000 66%) 1.6668
17m 31s (- 6m 22s) (55000 73%) 1.5800
19m 6s (- 4m 46s) (60000 80%) 1.5319
20m 41s (- 3m 11s) (65000 86%) 1.4329
22m 18s (- 1m 35s) (70000 93%) 1.3761
23m 54s (- 0m 0s) (75000 100%) 1.3576


In [31]:
evaluateRandomly(encoder1, decoder1)

> он безжалостен .
= he s ruthless .
< he is a . <EOS>

> я удовлетворенныи .
= i m contented .
< i m a . <EOS>

> ты теперь отец .
= you re a father now .
< you re a to . <EOS>

> мы немного устали .
= we re a little tired .
< we re a little tired . <EOS>

> ты такои романтичныи .
= you re so romantic .
< you re so stupid . <EOS>

> он выпивает .
= he s a bit of a drunkard .
< he is a . . <EOS>

> вы обманщица .
= you re a fraud .
< you re a . <EOS>

> он очень ко мне добр .
= he s very kind to me .
< he is very to to me . <EOS>

> они обе улыбаются .
= they re both smiling .
< they re both smiling . <EOS>

> мы ищем сокровище .
= we re looking for the treasure .
< we re looking for a job . <EOS>



На уровне GRU. Попробуем два слоя.

# LSTM с двумя рекуррентными слоями.

In [32]:
hidden_size = 256
encoder1 = EncoderRNN(input_lang.n_words, hidden_size, rnn_type='lstm', rnn_n=2).to(device)
decoder1 = DecoderRNN(hidden_size, output_lang.n_words, rnn_type='lstm', rnn_n=2).to(device)

trainIters(encoder1, decoder1, 75000, print_every=5000)

2m 14s (- 31m 23s) (5000 6%) 3.3574
4m 23s (- 28m 35s) (10000 13%) 3.1298
6m 33s (- 26m 12s) (15000 20%) 3.0550
8m 42s (- 23m 57s) (20000 26%) 2.8107
10m 54s (- 21m 49s) (25000 33%) 2.7191
13m 7s (- 19m 41s) (30000 40%) 2.6775
15m 20s (- 17m 31s) (35000 46%) 2.5867
17m 32s (- 15m 20s) (40000 53%) 2.5173
19m 45s (- 13m 10s) (45000 60%) 2.4623
21m 59s (- 10m 59s) (50000 66%) 2.4053
24m 13s (- 8m 48s) (55000 73%) 2.3255
26m 26s (- 6m 36s) (60000 80%) 2.2665
28m 40s (- 4m 24s) (65000 86%) 2.2136
30m 54s (- 2m 12s) (70000 93%) 2.1469
33m 8s (- 0m 0s) (75000 100%) 2.0745


In [33]:
evaluateRandomly(encoder1, decoder1)

> я ужасно голоден .
= i m terribly hungry .
< i m a . . <EOS>

> не уверен что согласен с тобои .
= i m not sure i agree with you .
< i m not sure you re ready . <EOS>

> я рада что мы победили .
= i m happy we won .
< i m glad i ve . . <EOS>

> я собираюсь кое что купить тому .
= i m going to buy tom something .
< i m going to tom tom tom tom <EOS>

> я редко опаздываю в школу .
= i m seldom late for school .
< i m going to to my . . <EOS>

> простите но вы ошибаетесь .
= i m sorry but you re wrong .
< i m sorry i i you you . <EOS>

> вы виновны в убиистве .
= you are guilty of murder .
< you re the one of . . <EOS>

> я почти уверен что том победит .
= i m pretty sure tom will win .
< i m sure tom will be . . <EOS>

> вы очень хорошо рисуете .
= you re very good at drawing .
< you re very good at . . <EOS>

> мы не планируем ехать вместе .
= we re not planning to go together .
< we re not ready to help . <EOS>

